## Environment: 

Python 3.9

## Requirements: 

pip install _gradio opencv-python pillow numpy pillow-lut_

In [20]:
import gradio as gr
from PIL import Image, ImageEnhance
from pillow_lut import load_cube_file
import os
import numpy as np

def adjust_tint(image, factor):
    img_array = np.array(image).astype(float)
    img_array[:,:,1] = np.clip(img_array[:,:,1] * factor, 0, 255)
    return Image.fromarray(img_array.astype('uint8'))

def adjust_white_balance(image, kelvin):
    if kelvin <= 6600:
        red = 255
        green = 99.4708025861 * np.log(kelvin / 100) - 161.1195681661
        blue = 138.5177312231 * np.log(kelvin / 100 - 10) - 305.0447927307 if kelvin > 1900 else 0
    else:
        red = 329.698727446 * ((kelvin / 100 - 60) ** -0.1332047592)
        green = 288.1221695283 * ((kelvin / 100 - 60) ** -0.0755148492)
        blue = 255

    r_multiplier = max(0, min(255, red)) / 255
    g_multiplier = max(0, min(255, green)) / 255
    b_multiplier = max(0, min(255, blue)) / 255

    img_array = np.array(image).astype(float)
    img_array[:,:,0] = np.clip(img_array[:,:,0] * r_multiplier, 0, 255)
    img_array[:,:,1] = np.clip(img_array[:,:,1] * g_multiplier, 0, 255)
    img_array[:,:,2] = np.clip(img_array[:,:,2] * b_multiplier, 0, 255)

    return Image.fromarray(img_array.astype('uint8'))

def process_image(input_image, lut_option, brightness, contrast, white_balance, tint, saturation):
    if input_image is None:
        return None
    
    # Apply LUT
    lut_path = f"luts/{lut_option}.cube"
    lut = load_cube_file(lut_path)
    result = input_image.filter(lut)
    
    # Adjust brightness
    enhancer = ImageEnhance.Brightness(result)
    result = enhancer.enhance(brightness)
    
    # Adjust contrast
    enhancer = ImageEnhance.Contrast(result)
    result = enhancer.enhance(contrast)
    
    # Adjust white balance
    result = adjust_white_balance(result, white_balance)
    
    # Adjust tint
    result = adjust_tint(result, tint)
    
    # Adjust saturation
    enhancer = ImageEnhance.Color(result)
    result = enhancer.enhance(saturation)
    
    return result

# Get list of LUT files
lut_files = [f.split('.')[0] for f in os.listdir('luts') if f.endswith('.cube')]

# Create Gradio interface
iface = gr.Interface(
    fn=process_image,
    inputs=[
        gr.Image(type="pil", label="Upload Image"),
        gr.Dropdown(choices=lut_files, label="Select LUT Filter", value=lut_files[0]),
        gr.Slider(minimum=0.5, maximum=1.5, value=1.0, step=0.01, label="Brightness"),
        gr.Slider(minimum=0.5, maximum=1.5, value=1.0, step=0.01, label="Contrast"),
        gr.Slider(minimum=2500, maximum=8500, value=5500, step=100, label="White Balance (Kelvin)"),
        gr.Slider(minimum=0.5, maximum=1.5, value=1.0, step=0.01, label="Tint"),
        gr.Slider(minimum=0, maximum=2, value=1.0, step=0.01, label="Saturation")
    ],
    outputs=gr.Image(type="pil", label="Processed Image"),
    title="Fxck Adobe",
    live=True,
    theme=gr.themes.Monochrome(),
    allow_flagging="never"
)

# Launch the interface
iface.launch()

Running on local URL:  http://127.0.0.1:7878

To create a public link, set `share=True` in `launch()`.
